In [1]:
##### -*-coding:utf-8 -*-
# import all the libraries 
# python==3.8; jupyterlab==3.0.12; lumicks.pylake==0.8.1; matplotlib==3.3.4; more-itertools==8.7.0;
# npTDMS==1.1.0; numpy==1.20.1; opencv-python==4.5.1.48; pandas==1.2.3; scipy==1.6.1; tifffile==2021.3.5
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from __future__ import division
import ruptures as rpt
import pwlf
from scipy import interpolate
from __future__ import division
from more_itertools import chunked
%matplotlib widget

# In order to find optimal parameters for the rupture algrithm, we tried different penelty value based on the times of sigma, and compare the results of detected change-point and residuals
# To this end, we first generate linear segments with similar vertical resolutuion (thousands of basepairs) and time resolution (20HZ), and add gussain noise with a standard deveation of sigma=20
# In oder to calculate the residual, an interpolate performance is applied to get the estimate of detected change-point 

In [2]:
# generate linear segments data
time = np.linspace(0,220,4400)

y1 = np.linspace(10000,9500,400)
y2 = np.linspace(9500,8500,500)
y3 = np.linspace(8500,8500,500)
y4 = np.linspace(8500,8000,700)
y5 = np.linspace(8000,5000,1100)
y6 = np.linspace(5000,5000,200)
y7 = np.linspace(5000,6500,400)
y8 = np.linspace(6500,7000,200)
y9 = np.linspace(7000,7300,100)
y10 = np.linspace(7300,7300,300)
e = [y1,y2,y3,y4,y5,y6,y7,y8,y9,y10]
raw_signal = [y for x in e for y in x]
raw_signal = np.array(raw_signal)
# print(len(signal))

# # generate sin wave data
# x = np.linspace(0, 10, num=10000)
# y = np.sin(x * np.pi / 2)
# # add noise to the data
# y = np.random.normal(0, 0.1, 10000) + y

# # plot the results
# plt.figure()
# plt.plot(x, y, 'o')
# plt.show()

# add noise to the data
sigma = 20
noise = np.random.normal(0, sigma, 4400) 
signal = raw_signal + noise

# plot the results
plt.figure(figsize=(5,4))

plt.plot(time, signal, label = "Guassian Noise")
plt.plot(time, raw_signal,label = "Simulated Data")

plt.xlabel("Time Series")
plt.ylabel("Simulated Signal")
plt.title("Simulated Data and PLRupture Detection")
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# this is to save the simulated data in txt and excel for further use
filename = r"C:\Users\KTS260\Desktop\simulated_data"
txtpath = filename + '.txt'
xlspath = filename + '.xlsx'

txt_data = open(txtpath, "w")
column = np.column_stack((time, signal)) 
np.savetxt(txt_data, column,delimiter=" , ")
txt_data.close()

# Save all the simulated data in an excel file
writer = pd.ExcelWriter(xlspath)
data = {'time':time,
        'signal':signal}
df = pd.DataFrame(data)
df.to_excel(writer)
writer.save()

In [156]:
# PLRupture is an improved changing point detection model based on rupture and piece-linear wise function
def PLRupture(signal,time,sigma):
    time_ROI = time
    basepairs_ROI = signal

# Apply proper model & searching method to work on change point detection
    model = "l2"  # "l1", "rbf", "linear", "normal", "ar",...

# For faster predictions, one can modify the jump parameter during initialization. The higher it is, the faster the prediction is achieved (at the expense of precision).
# There are plenty of models can be used for change point detection; but only rpt.Pelt seems to work with a algo.predict(pen=)
    algo = rpt.Pelt(model=model, jump=1).fit(signal)

# In our case, the number of change points is unknown, we need to specify a threshold on the residual norm using epsilon or a penalty using the pen parameter.
# n = number of samples
# sigma = noise standard deviation; the bigger the sigma, the less break points
# dim = dimention, in this case: dim = 1
    n = len(signal)
    dim = 1
# https://centre-borelli.github.io/ruptures-docs/user-guide/detection/bottomup/#:~:text=In%20the%20situation,n%20*%20sigma%20**%202)
# penelty constant = np.log(n) * dim * sigma ** 2
    my_bkps = algo.predict(pen=np.log(n) * dim * sigma ** 2)

# trace back to time-basepair based on index
    time_bkp = np.zeros(len(my_bkps)-1)
    bps_bkp = np.zeros(len(my_bkps)-1)

# to exclude the last value
    time_bkp = time_ROI[my_bkps[0:-1]]
    bps_bkp = basepairs_ROI[my_bkps[0:-1]]

# to add the first value and last value to array
    time_bkp = np.insert(time_bkp,0,time_ROI[0])
    time_bkp = np.append(time_bkp,time_ROI[-1])
    bps_bkp = np.insert(bps_bkp,0,basepairs_ROI[0])
    bps_bkp = np.append(bps_bkp,basepairs_ROI[-1])
    
# # initialize piecewise linear fit with your x and y data
#     my_pwlf = pwlf.PiecewiseLinFit(time_ROI, basepairs_ROI)

# # Uses L-BFGS-B optimization to find the location of breakpoints from a guess of where breakpoint locations should be.
#     time_bkp = my_pwlf.fit_guess(time_bkp)

#     from scipy import interpolate
#     func_2 = interpolate.interp1d(time_ROI, basepairs_ROI,kind='slinear',fill_value="extrapolate")
#     bps_bkp = func_2(time_bkp)

# # bks = np.append(time_bkp,bps_bkp, axis=-1).reshape((2,len(my_bkps)+1)).T
#     duration_time = np.diff(time_bkp)
#     processivity_event = np.diff(bps_bkp)
#     speed = processivity_event / duration_time
#     bound_duration = time_bkp[-1] - time_bkp[0]
#     bound_processivity = bps_bkp[-1] - bps_bkp[0]
    return time_bkp,bps_bkp


In [157]:
time_bkp,bps_bkp = PLRupture(signal,time,75*sigma)

In [158]:
# plot the simulated data and change-point detecting results
plt.figure(figsize=(5,4))

plt.plot(time, signal, label = "Guassian Noise")
plt.plot(time, raw_signal,label = "Simulated Data")
plt.scatter(time_bkp,bps_bkp,marker = "*",c = 'y', s = 100, label = "change-point")

plt.xlabel("Time Series")
plt.ylabel("Simulated Signal")
plt.title("Simulated Data and PLRupture Detection")
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [159]:
# print the number of detected change-points
print(len(bps_bkp))
print(len(time_bkp))

#
real_bkp = np.array([10000,9500,8500,8500,8000,5000,5000,6500,7000,7300,7300])
real_tkp = np.array([0,20,45,70,105,160,170,190,200,205,220])

from scipy import interpolate
func = interpolate.interp1d(time_bkp,bps_bkp,kind='slinear',fill_value="extrapolate")
predic_bkp = func(real_tkp)
residual = np.sum((predic_bkp-real_bkp)**2)

# print((predic_bkp-real_bkp)**2)
print(predic_bkp)
print(residual)

12
12
[10012.91431951  9423.60796188  8696.16492863  8417.69656809
  7818.24175295  5543.65153656  5546.34094429  6464.85243493
  6980.3437411   7057.82282666  7290.26008333]
738704.9073503715


In [160]:
# plot the detected change-points as a function of number of sigma
plt.figure(figsize=(5,4))

sigma_arr = np.array([1,2,5,10,20,50,75,100,500])
detected_change_points = np.array([193,128,72,44,29,16,12,10,3])

plt.scatter(sigma_arr,detected_change_points,marker = "*", s = 100, label = "number of Detected Change-Points")

plt.xlabel("Number of Sigma")
plt.ylabel("Number of Detected Change-points")
plt.title("Detected Change-points as a Function of Sigma")
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
# plot the residuals as a function of number of sigma
plt.figure(figsize=(5,4))

sigma_arr = np.array([1,2,5,10,20,50,75,100,500])
residual_arr = np.array([8390,7498,25832,40925,92032,664496,347610,1350853,12626854])
residual_norm = residual_arr/(np.max(residual_arr))

plt.scatter(sigma_arr,residual_norm,marker = "^", s = 100, label = "number of Detected Change-Points")# plt.scatter(sigma_arr,detected_change_points,marker = "*", s = 100, label = "Normalized Residuals")

plt.xlabel("Number of Sigma")
plt.ylabel("Normalized Residual")
plt.title("Normalized Residual as a Function of Sigma")
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …